## High-frequency trading in a limit order book by Marco Avellaneda & Sasha Stoikov

The goal of this notebook is to replicate the seminal paper of Avellaneda & Stoikov on High-frequency market making on limit order book. The paper discusses inventory risk face by a market maker while while submitting bid and ask quotes to the limit order. This is due to the diffusive nature of the stock’s mid-price and a transactions risk due to a Poisson arrival of market buy and sell orders. Therefore the authors suggest a two step procedure to maximize the expected utility of the agent. First is to compute a personal indifference valuation for the stock based on his current inventory risk. Second is to calibrate his bid and ask quotes to the market limit order book based on his personal indifference valuation.

### Optimal Model

Given the following parameters - 

- $s$ : current mid-price 
- $q$ : current inventory of the stock
- $\sigma$ : volatility
- $T$ : terminal time (normalized to 1)
- $t$ : current time fraction (part of T/dt)
- $\gamma$ : inventory risk aversion (0 is no risk aversion, 1 is full risk aversion)
- $\kappa$ : order book liquidity density (related to arrival rate of orders)

Find the following : 

#### Reservation Price
$$r(s,q,t) = s - q\gamma \sigma^2(T-t)$$

#### Optimal Bid/Ask Spread

$$\delta^a + \delta^b = \gamma \sigma^2(T-t)+\frac{2}{\gamma}\text{ln}(1+\frac{\gamma}{\kappa})$$

Now place the following limit orders :

- BUY Order @ $r(s,q,t) - (\delta^a + \delta^b)/2$
- SELL Order @ $r(s,q,t) + (\delta^a + \delta^b)/2$

### Trading Intensity
The intensity at which agents orders are executed will depend only on th distance of his quotes to the mid-price i.e. $\lambda^b(\delta^b)$ for the arrival of sell orders and $\lambda^a(\delta^a)$ for the arrival of buy orders. It is modeled using the following : 
$$\lambda(\delta) = Ae^{-\kappa\delta}$$
where, $A=\Lambda/\alpha$ and $\Lambda$ can be estimated by dividing the total volume traded over a day by the average
size of market orders on that day.

In [8]:
import time
import math
import random

import numpy as np
import matplotlib.pyplot as plt

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot

output_notebook()

Loading BokehJS ...

In [2]:
# Parameters for mid price simulation:
S0 = 100.0 # initial price
T = 1.0 # time
sigma = 2 # volatility
M = 20000 # number of time steps
dt = T/M # time step
Sim = 100 # number of simulations
gamma = 0.1 # risk aversion
k = 1.5 #
A = 140 # order frequency
I = 1 # stocks
M_float = float(M)
sqrt_dt = math.sqrt(dt)

# Results:
AverageSpread = []
Profit = []
Std = []

In [3]:
for i in range(1, Sim+1):
    # reservation price:
    # r(s,t) = s - q * gamma * sigma**2 * (T-t)
    S = np.zeros((M+1, I))
    Bid = np.zeros((M+1, I))
    Ask = np.zeros((M+1, I))
    ReservPrice = np.zeros((M+1, I))
    spread = np.zeros((M+1, I))
    deltaB = np.zeros((M+1, I))
    deltaA = np.zeros((M+1, I))
    q = np.zeros((M+1, I))
    w = np.zeros((M+1, I))
    equity = np.zeros((M+1, I))

    S[0] = S0
    ReservPrice[0] = S0
    Bid[0] = S0
    Ask[0] = S0
    spread[0] = 0
    deltaB[0] = 0
    deltaA[0] = 0
    q[0] = 0 # position
    w[0] = 0 # wealth
    equity[0] = 0
    for t in range(1, M+1):
        z = np.random.standard_normal(I)# dW
        S[t] = S[t-1] + sigma * sqrt_dt * z# new S value
        ReservPrice[t] = S[t] - q[t-1] * gamma * (sigma ** 2) * (T - t/M_float)# Mid Price estimation
        spread[t] = gamma * (sigma ** 2) * (T - t/M_float) + (2/gamma) * math.log(1 + (gamma/k)) # Spread estimation
        Bid[t] = ReservPrice[t] - spread[t]/2.# bid
        Ask[t] = ReservPrice[t] + spread[t]/2.# ask

        deltaB[t] = S[t] - Bid[t]# difference to price
        deltaA[t] = Ask[t] - S[t]# difference to price

        lambdaA = A * np.exp(-k * deltaA[t])
        ProbA = 1 - np.exp(-lambdaA * dt)
        fa = random.random()

        lambdaB = A * np.exp(-k * deltaB[t])
        ProbB = 1 - np.exp(-lambdaB * dt)
        fb = random.random()

        if ProbB > fb and ProbA < fa:# buy market order filling our limit order
            q[t] = q[t-1] + 1# position
            w[t] = w[t-1] - Bid[t]# wealth

        if ProbB < fb and ProbA > fa:# sell market order filling our limit order
            q[t] = q[t-1] - 1# position
            w[t] = w[t-1] + Ask[t]# wealth

        if ProbB < fb and ProbA < fa:# no order
            q[t] = q[t-1]# position
            w[t] = w[t-1]# wealth

        if ProbB > fb and ProbA > fa:# both sides order
            q[t] = q[t-1]# position
            w[t] = w[t-1] - Bid[t]# wealth
            w[t] = w[t] + Ask[t]

        equity[t] = w[t] + q[t] * S[t]
    AverageSpread.append(spread.mean())
    Profit.append(equity[-1])
    Std.append(np.std(equity))

In [17]:
print("                   Results              ")
print("----------------------------------------")
print("%14s %21s" % ('statistic', 'value'))
print(40 * "-")
print("%14s %20.5f" % ("Average spread :", np.array(AverageSpread).mean()))
print("%16s %20.5f" % ("Profit :", np.array(Profit).mean()))
print("%16s %20.5f" % ("Std(Profit) :", np.array(Std).mean()))

x = np.linspace(0., T, num=(M+1))
# Create Bokeh figure objects
fig1 = figure(title='Prices', width=800, height=400, x_axis_type='datetime')
fig1.line(x, S.reshape(1,-1)[0], line_width=1., legend_label='Mid-market price', color='navy')
fig1.line(x, ReservPrice.reshape(1,-1)[0], line_width=1., legend_label='Indifference price', color='orange')
fig1.circle(x, Ask.reshape(1,-1)[0], size=1, line_color="red", fill_color="white", fill_alpha=0.5, legend_label='Priced ask')
fig1.circle(x, Bid.reshape(1,-1)[0], size=1, line_color="green", fill_color="white", fill_alpha=0.5, legend_label='Priced bid')
fig1.grid.grid_line_alpha = 0.3
fig1.legend.location = 'top_right'
fig1.xaxis.axis_label = 'Time'
fig1.yaxis.axis_label = 'Stock Price'


fig2 = figure(title='Position', width=800, height=400, x_axis_type='datetime')
fig2.line(x, q.reshape(1,-1)[0], line_color='green', line_width=1., legend_label='q')
fig2.grid.grid_line_alpha = 0.3
fig2.legend.location = 'top_right'
fig2.xaxis.axis_label = 'Time'
fig2.yaxis.axis_label = 'Position'


fig3 = figure(title='Histogram', width=400, height=400)
fig3.quad(top=np.histogram(np.array(Profit), bins=100)[0], bottom=0, left=np.histogram(np.array(Profit), bins=100)[1][:-1], right=np.histogram(np.array(Profit), bins=100)[1][1:], fill_color='red', line_color='black', legend_label='Inventory Strategy') 
fig3.grid.grid_line_alpha = 0.3
fig3.legend.location = 'top_right'
fig3.xaxis.axis_label = 'pnl'
fig3.yaxis.axis_label = 'N'

fig4 = figure(title='Equity', width=400, height=400, x_axis_type='datetime')
fig4.line(x, equity.reshape(1,-1)[0], line_width=1., legend_label='equity', color='navy')
fig4.grid.grid_line_alpha = 0.3
fig4.legend.location = 'top_right'
fig4.xaxis.axis_label = 'Time'
fig4.yaxis.axis_label = 'Price'

# Create a grid layout of figures
grid = gridplot([[fig3, fig4]])
show(fig1)
show(fig2)
# Show the plot
show(grid)

                   Results              
----------------------------------------
     statistic                 value
----------------------------------------
Average spread :              1.49069
        Profit :             64.36585
   Std(Profit) :             18.71813
